# 필수 모듈 임포트

In [1]:
# 필수 모듈 임포트
import requests
import re
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import json

C:\Users\user\AppData\Local\Temp\ipykernel_24176\1814035643.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# 크롤링 함수 정의

## json 저장 및 로드 함수

In [11]:
def save_to_json(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        
def read_from_json(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return json.load(file)

## 인덱스 호출 함수

In [12]:
def load_last_index(filename):
    try:
        with open(filename, 'r') as file:
            data = json.load(file)
            return data.get('last_completed_page') or data.get('last_processed_index')
    except FileNotFoundError:
        return 0  # 파일이 없을 경우 0부터 시작

## - 의사 정보 크롤링

In [13]:
def scrape_doctor_profiles(max_pages, start_page=0):
    base_url = 'https://kin.naver.com/people/expert/index.naver?type=DOCTOR&page={}'
    doctor_info = []

    for page in tqdm(range(start_page, max_pages + 1)):
        url = base_url.format(page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 각 의사의 상세 페이지 링크와 정보를 찾아 리스트에 추가
        for item in soup.select('.pro_list li'):
            doctor_link_tag = item.find('h5').find('a')
            if doctor_link_tag:
                doctor_name = doctor_link_tag.text.strip()  # 닥터 이름 추출
                doctor_id = doctor_link_tag['href'].split('u=')[1]  # 사용자 ID 추출
                specialty_tag = item.find('h6')
                specialty = specialty_tag.text.strip() if specialty_tag else '정보 없음' # 전문과목 추출
                affiliation_tag = item.find('th', string='소속기관')
                affiliation = affiliation_tag.find_next('td').text.strip() if affiliation_tag else '정보 없음' # 소속기관 추출
                answer_count_tag = item.find('th', string='총 답변')
                answer_count = int(answer_count_tag.find_next('td').text.strip().replace(',', '')) if answer_count_tag else 0 # 총 답변 수 추출

                # 의사 정보를 딕셔너리로 저장
                doctor_info.append({
                    'doctor_id': doctor_id,
                    'doctor_name': doctor_name,
                    'specialty': specialty,
                    'total_answers': answer_count,
                    'affiliation': affiliation
                })
        save_to_json({'last_completed_page': page}, 'last_page.json')

    return doctor_info

In [5]:
# # 결과값 예시
# doctor_profiles = scrape_doctor_profiles(1)
# doctor_profiles[0]

100%|██████████| 2/2 [00:00<00:00,  2.64it/s]


{'doctor_id': 'OnBDJZV4kCt9ZeKxepFuNfUzZhlGzWcwpGWftWgpkvE%3D',
 'doctor_name': '정현화',
 'specialty': '늘행복요양병원',
 'total_answers': 84856,
 'affiliation': '대한의사협회'}

## - 질문 id 크롤링(의사별)

In [22]:
## cheol(0209): print(page) 넣어둠

def scrape_info(doctor_id, total_answers):
    max_pages = total_answers // 20 + 1 if total_answers % 20 else total_answers // 20
    base_url = 'https://kin.naver.com/userinfo/expert/answerList.naver?u={user_id}&page={page}'
    all_info = {}

    for page in tqdm(range(1, max_pages + 1), desc=f"Scraping {doctor_id}"):
        url = base_url.format(user_id=doctor_id, page=page)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        # 원하는 정보를 추출하는 정규 표현식 패턴 정의
        pattern = r'<a href="/qna/detail\.naver\?d1id=\d+&dirId=\d+&docId=(\d+)"'
        # 정규 표현식으로 링크 찾기
        matches = re.findall(pattern, response.text)
        # 날짜 데이터를 추출하여 리스트로 저장
        dates = [date.text.strip() for date in soup.select('.t_num.tc')]
        # 찾아진 링크와 날짜를 all_info에 추가
        
        print(page)
        
        for i in range(len(matches)):
            doc_id = matches[i]
            date = dates[i] if i < len(dates) else '날짜 없음'
            if doctor_id not in all_info:
                all_info[doctor_id] = []
            all_info[doctor_id].append({'doc_id': doc_id, 'date': date})
    return all_info


In [7]:
# doc_ids = scrape_info(doctor_profiles[0]['doctor_id'],doctor_profiles[0]["total_answers"])
# doc_ids

## 모든 doc_id 추출 함수

In [35]:
## cheol: i 2005일때 break 시켰고 last_index.json 2000으로 해뒀음

def scrape_doc_ids(doctor_profiles, start_index=0):
    try:
        last_processed_index = read_from_json('last_index.json')['last_processed_index']
    except FileNotFoundError:
        last_processed_index = start_index
        
    doc_ids_data = {}
    for i, profile in tqdm(enumerate(doctor_profiles[start_index:], start=start_index), total=len(doctor_profiles[start_index:])):
        doctor_id = profile['doctor_id']
        doc_ids = scrape_info(doctor_id, profile['total_answers'])
        doc_ids_data[doctor_id] = doc_ids
        save_to_json({'last_processed_index': i, 'doc_ids_data': doc_ids_data}, 'doc_ids_data_cheol.json')
        print('내가',i)
        if i == 2005:
            break
    return doc_ids_data

In [9]:
# doc_ids = scrape_doc_ids(doctor_profiles)
# doc_ids

## 질문 답변 크롤링

In [113]:
# 스크래핑 함수 정의

## cheol: time.sleep 주석처리함 2개다 주석함
## 추가로 doc_id['doc_id'] 이부분 걍 doc_id로함 ( doc_ids는 doc_id만 있는 리스트로 해둠 ㅋ)

def scrape_details(doc_ids):
    base_url = 'https://kin.naver.com/qna/detail.naver?d1id=7&dirId=70201&docId={}'
    head = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6',
    'Cache-Control': 'max-age=0',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36',
    'Upgrade-Insecure-Requests': '1',}
    retry_delays = [5, 10, 20]
    
    # 데이터를 저장할 빈 리스트 생성
    title_list = []
    question_list = []
    answer_list = []

    # tqdm을 사용하여 진행 상황을 시각화
    for doc_id in tqdm(doc_ids):
        attempt = 0
        while attempt <= len(retry_delays):
            try:
                url = base_url.format(doc_id)
                r = requests.get(url, headers=head)
                r.raise_for_status()
                bs = BeautifulSoup(r.text, 'html.parser')

                title_data = bs.select_one('.title') # 질문 제목
                title = title_data.text.strip() if title_data else None
                
                question_data = bs.select_one('.c-heading__content') # 질문 내용
                question = question_data.text.strip() if question_data else None
                
                answer_data = bs.select_one('.se-main-container') # 답변
                answer = answer_data.text.strip() if answer_data else None

                title_list.append(title)
                question_list.append(question)
                answer_list.append(answer)
                break

            except requests.exceptions.HTTPError as e:
                if e.response.status_code == 429:  # Too Many Requests
                    if attempt == len(retry_delays):
                        tqdm.write(f"Failed to scrape doc_id {doc_id} after multiple attempts.")
                        break  # 최대 재시도 횟수에 도달하면 실패로 간주
                    tqdm.write(f"Rate limit reached. Retrying in 1 seconds...")
                    #time.sleep(1)  # 지정된 시간만큼 대기
                    attempt += 1
                else:
                    tqdm.write(f"Failed to scrape doc_id {doc_id['doc_id']}: {e}")
                    break  # 다른 유형의 HTTP 에러는 재시도하지 않음
            except requests.exceptions.RequestException as e:
                tqdm.write(f"Failed to scrape doc_id {doc_id['doc_id']}: {e}")
                break  # 네트워크 문제 등 다른 예외에 대한 처리
            #time.sleep(1)  # 다음 요청까지의 기본 지연 시간


    data = {'title': title_list, 'question': question_list, 'answer': answer_list}
    return pd.DataFrame(data)

# ⭐️모든 doc_id 추출⭐️

In [11]:
# all_doctor_profiles 추출
max_pages = 526
last_page = load_last_index('last_page.json')
doctor_profiles = scrape_doctor_profiles(max_pages, start_page=last_page)

# json 저장
doctor_profiles_filename = "all_doctor_profiles.json"
save_to_json(doctor_profiles, doctor_profiles_filename)
print(f"Data saved to {doctor_profiles_filename}")

100%|██████████| 526/526 [02:56<00:00,  2.98it/s]

Data saved to all_doctor_profiles.json


In [36]:
# doc_ids 추출
last_index = load_last_index('last_index.json')
doctor_profiles = read_from_json("all_doctor_profiles.json")
doc_ids_data = scrape_doc_ids(doctor_profiles, start_index=last_index) # json 저장

print(f"Data saved to doc_ids_data.json")

  0%|          | 0/3259 [00:00<?, ?it/s]

Scraping ukc8l8SA5jqtLV7IARW7hB0vYWRb6jsHZ7mIMvZciwE%3D: 100%|██████████| 1/1 [00:00<00:00,  1.16it/s]
  0%|          | 1/3259 [00:00<47:02,  1.15it/s]

1
내가 2000


1


2


...

35


36


37


Scraping u5pVm9ZblZ3h87jlDUOfq2ZciGzXqqHqVs3x1k8rfgs%3D: 100%|██████████| 38/38 [00:30<00:00,  1.25it/s]
  0%|          | 5/3259 [00:49<8:58:17,  9.93s/it]

38
내가 2005
Data saved to doc_ids_data.json


In [111]:
# doc_id['doc_id']
doc_ids=[]
for i in doc_ids_data:
    for j in doc_ids_data[i]:
        for k in doc_ids_data[i][j]:
            
            doc_ids.append(k['doc_id'])
            
            
        
            
            

In [114]:
# 개당 0.693초
scrape_details(doc_ids)

100%|██████████| 860/860 [10:02<00:00,  1.43it/s]


,title,question,answer
0,척추층만증 치료 될까요?,제가 경찰 희망하는 16살 남자입니다 제가 5개월전쯤 왼쪽으로 5도 앞쪽으로 5에서...,안녕하세요. 대한의사협회·네이버 지식iN 상담의사 조남익 입니다.​현재 나이가 16...
1,축구하다가 접질렸습니다. 축구하다가 상대방이 몸으로 밀다가 같이 넘어지면서 발목이 ...,축구하다가 접질렸습니다.축구하다가 상대방이 몸으로 밀다가 같이 넘어지면서 발목이 안...,안녕하세요. 대한의사협회·네이버 지식iN 상담의사 조남익 입니다.발목을 접질린 후 ...
2,손에 힘이 안들어가요,한 3일정도됬나? 자고일어나면 손에 힘이 안들어가면서 안구부러지는데 굳은건지한쪽손은...,안녕하세요. 대한의사협회·네이버 지식iN 상담의사 조남익 입니다.​자고 일어나신후 ...
3,신경 염증으로 꽤 오래 치료받았는데 낫지가 않네요..,약 6개월정도 목에 염증이 있어서 신경치료를 받았습니다. 탁구를 좀 무리하게 친 게...,안녕하세요. 대한의사협회·네이버 지식iN 상담의사 조남익 입니다.​현재 증상은 목 ...
4,의학적으로 목길이가 짧아 질수도 있나요?,목이 두꺼워져서 짧아 보인다거나 거북목이라 목이 휘어서 짧아 보일순 잇어도의학적으로...,안녕하세요. 대한의사협회·네이버 지식iN 상담의사 조남익 입니다.​나이가 들면서 목...
...,...,...,...
855,유럽여행 요도염,요도염 증상 ) 진물나고 소변 볼 때 타는 느낌이 나는데 여행은 시작한지 얼마 안됐...,안녕하세요. 하이닥-네이버 지식iN 상담의 장창식 입니다.​여행 중에 요도염으로 고...
856,고추에서 분비물같은게나와요,자다가인나면 속옷에뭍어있어요 노란색으로 진물인가요????사람들이 요도염같다는데 요도...,안녕하세요. 하이닥-네이버 지식iN 상담의 장창식 입니다.​검사를 진행하지 않고 질...
857,포경수술 운동,포경수술을12월 9일에 했습니다.근데 제가 24일에 무조건 해야만하는 축구 시합이 ...,안녕하세요. 하이닥-네이버 지식iN 상담의 장창식 입니다.​포경수술 후 축구와 같은...
858,정관수술 ?은지 4개월정도 지났는데 해외출장 관계로성관계를 한번도 하지않았습니다병원...,None,안녕하세요. 하이닥-네이버 지식iN 상담의 장창식 입니다.​정관 수술 후 15회 이...


In [ ]:
if os.path.isdir("./output_data") == False:
    os.mkdir("./output_data")


head = { 'User-Agent':
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}

for dirId in detailed_category:
  data_dict= {}

  for doc_id in detailed_category[dirId]:
      url = "https://kin.naver.com/qna/detail.naver?d1id=7&dirId=7010101&docId={}".format(doc_id)
      r = requests.get(url, headers=head)
      bs = BeautifulSoup(r.text, 'html.parser')

      tmp_dict={}

      title_data = bs.select('.title')
      title = title_data[0].text.strip()
      tmp_dict['title']= title

      date = bs.select('.c-userinfo__info')
      date_ = date[0].text.replace("작성일", "")
      tmp_dict['date_']= date_

      question_data = bs.select('.c-heading__content')
      question = question_data[0].text.strip()
      tmp_dict['question']= question

      answer_data = bs.select('.se-main-container')
      answer = answer_data[0].text.strip()
      tmp_dict['answer']= answer

      data_dict[doc_id]=tmp_dict


  # dirId별로 json 파일로 저장
  data_json = json.dumps(data_dict)
  with open(f'./output_data/data_{dirId}.json', 'w') as json_file:
    json_file.write(data_json)


## 의사 1명에 대한 샘플 추출

In [ ]:
doctor_profiles = scrape_doctor_profiles(1)
doctor_profiles[0], doctor_profiles

In [ ]:
doc_ids_info = scrape_info(doctor_profiles[0]['doctor_id'],doctor_profiles[0]['total_answers'])
doc_ids_info, doc_ids_info[doctor_profiles[0]['doctor_id']]

In [ ]:
df = scrape_details(doc_ids_info[doctor_profiles[0]['doctor_id']])

In [ ]:
for doctor in doctor_profiles:
    doctor_id = doctor['doctor_id']
    total_answers = doctor['total_answers']
    doc_ids_info = scrape_info(doctor_id, total_answers)
    if doctor_id in doc_ids_info:
        df = scrape_details(doc_ids_info[doctor_id])
        # 파일명에 의사 ID를 포함하여 각 의사별로 고유한 JSON 파일 생성
        filename = f'doctor_data_{doctor_id}.json'
        df.to_json(filename, orient='records', force_ascii=False, lines=True)
        break